In [ ]:
import pandas as pd
import numpy as np
import os
import requests

#############################################
# Optional: Groq API call for cleaning advice
#############################################
def get_groq_cleaning_instructions(data_summary):
    """
    Uses the Groq API to generate cleaning instructions based on a data summary.
    (Replace API_KEY and ensure the endpoint and model are correct per your Groq documentation.)
    """
    API_KEY = "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"  # Replace with your actual key
    API_URL = "https://api.groq.com/openai/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    prompt = (
        "Based on the following data summary, provide data cleaning and transformation steps "
        "so that the dataset is ready for visualization in Power BI. Ensure that missing values, "
        "date/time formats, numeric rounding, and data types are handled appropriately.\n\n"
        f"{data_summary}"
    )

    payload = {
        "model": "llama-3.3-70b-versatile",  # Use the model per your API reference
        "messages": [{"role": "user", "content": prompt}]
    }

    try:
        response = requests.post(API_URL, json=payload, headers=headers)
        if response.status_code == 200:
            instructions = response.json().get("choices", [{}])[0].get("message", {}).get("content", "")
            return instructions
        else:
            return f"Error {response.status_code}: {response.text}"
    except Exception as e:
        return f"Error calling Groq API: {e}"

#############################################
# Dynamic Data Cleaning Function
#############################################
def dynamic_clean_dataset(filepath):
    """
    Loads a dataset (Excel or CSV), dynamically inspects and cleans the data,
    and returns a cleaned DataFrame ready for Power BI.
    """
    # Load the dataset based on file extension
    try:
        if filepath.lower().endswith((".xlsx", ".xls")):
            df = pd.read_excel(filepath)
        elif filepath.lower().endswith(".csv"):
            df = pd.read_csv(filepath)
        else:
            print("Unsupported file format. Please use Excel or CSV.")
            return None
    except Exception as e:
        print("Error loading dataset:", e)
        return None

    print("Original Data Info:")
    print(df.info())

    # Process each column dynamically
    for col in df.columns:
        col_dtype = df[col].dtype

        if col_dtype == 'object':
            # Try converting to datetime if possible
            try:
                # If you know the format, specify it, e.g. format="%m/%d/%Y"
                df[col] = pd.to_datetime(df[col], errors='raise')
                print(f"Column '{col}' converted to datetime.")
            except Exception:
                # If conversion fails, treat as text and convert to categorical
                df[col] = df[col].astype('category')
                # If you plan to fill missing values, add a default category first.
                if "Unknown" not in df[col].cat.categories:
                    df[col] = df[col].cat.add_categories(['Unknown'])
                df[col] = df[col].fillna('Unknown')
                print(f"Column '{col}' set to categorical with missing filled as 'Unknown'.")
        elif np.issubdtype(col_dtype, np.number):
            # For numeric columns, fill missing values with the median.
            if df[col].isnull().any():
                median_val = df[col].median()
                df[col] = df[col].fillna(median_val)
                print(f"Numeric column '{col}' missing values filled with median ({median_val}).")
            # Round floats to 2 decimal places.
            if pd.api.types.is_float_dtype(df[col]):
                df[col] = df[col].round(2)
                print(f"Numeric column '{col}' rounded to 2 decimals.")
        else:
            # For any other types, try filling missing values with the mode.
            if df[col].isnull().any():
                mode_series = df[col].mode()
                fill_value = mode_series[0] if not mode_series.empty else "Unknown"
                # If the column is categorical, ensure the fill_value is a valid category.
                if hasattr(df[col], "cat"):
                    if fill_value not in df[col].cat.categories:
                        df[col] = df[col].cat.add_categories([fill_value])
                df[col] = df[col].fillna(fill_value)
                print(f"Column '{col}' missing values filled with mode/default ({fill_value}).")

    # After processing, check for any remaining missing values.
    missing_after = df.isnull().sum()
    print("Missing values after cleaning:")
    print(missing_after)

    # For any residual missing values, fill with a default value.
    df = df.fillna("Unknown")
    return df

#############################################
# Main Routine
#############################################
if __name__ == "__main__":
    # Specify the dataset path (adjust the path as needed)
    filepath = "Amazon Sale Report.csv"  # Change to your file path

    # Dynamically clean the dataset
    cleaned_df = dynamic_clean_dataset(filepath)
    # Check if cleaning was successful before proceeding
    if cleaned_df is None:
        print("Data cleaning failed.")
        exit()  # Exit the script if cleaning fails

    # (Optional) Generate a data summary for the Groq API
    data_summary = cleaned_df.describe(include='all').to_string()
    groq_instructions = get_groq_cleaning_instructions(data_summary)

    print("\n--- Groq API Cleaning Instructions ---")
    print(groq_instructions)
    print("--- End of Instructions ---\n")

    # Save the cleaned data to a CSV file (ready for Power BI)
    output_file = "Cleaned_Data_For_PowerBI.csv"
    cleaned_df.to_csv(output_file, index=False)
    print(f"Cleaned data saved to '{output_file}'.")

<ipython-input-1-465ce396b91b>:57: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


Original Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               128975 non-null  int64  
 1   Order ID            128975 non-null  object 
 2   Date                128975 non-null  object 
 3   Status              128975 non-null  object 
 4   Fulfilment          128975 non-null  object 
 5   Sales Channel       128975 non-null  object 
 6   ship-service-level  128975 non-null  object 
 7   Style               128975 non-null  object 
 8   SKU                 128975 non-null  object 
 9   Category            128975 non-null  object 
 10  Size                128975 non-null  object 
 11  ASIN                128975 non-null  object 
 12  Courier Status      122103 non-null  object 
 13  Qty                 128975 non-null  int64  
 14  currency            121180 non-null  object 
 15  Amount        

<ipython-input-1-465ce396b91b>:76: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='raise')


Column 'Order ID' set to categorical with missing filled as 'Unknown'.
Column 'Date' converted to datetime.
Column 'Status' set to categorical with missing filled as 'Unknown'.
Column 'Fulfilment' set to categorical with missing filled as 'Unknown'.
Column 'Sales Channel ' set to categorical with missing filled as 'Unknown'.
Column 'ship-service-level' set to categorical with missing filled as 'Unknown'.
Column 'Style' set to categorical with missing filled as 'Unknown'.
Column 'SKU' set to categorical with missing filled as 'Unknown'.
Column 'Category' set to categorical with missing filled as 'Unknown'.
Column 'Size' set to categorical with missing filled as 'Unknown'.


<ipython-input-1-465ce396b91b>:76: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='raise')
<ipython-input-1-465ce396b91b>:76: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='raise')
<ipython-input-1-465ce396b91b>:76: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='raise')
<ipython-input-1-465ce396b91b>:76: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-exp

Column 'ASIN' set to categorical with missing filled as 'Unknown'.
Column 'Courier Status' set to categorical with missing filled as 'Unknown'.
Column 'currency' set to categorical with missing filled as 'Unknown'.
Numeric column 'Amount' missing values filled with median (605.0).
Numeric column 'Amount' rounded to 2 decimals.
Column 'ship-city' set to categorical with missing filled as 'Unknown'.
Column 'ship-state' set to categorical with missing filled as 'Unknown'.
Numeric column 'ship-postal-code' missing values filled with median (500033.0).
Numeric column 'ship-postal-code' rounded to 2 decimals.
Column 'ship-country' set to categorical with missing filled as 'Unknown'.
Column 'promotion-ids' set to categorical with missing filled as 'Unknown'.
Column 'fulfilled-by' set to categorical with missing filled as 'Unknown'.
Column 'Unnamed: 22' set to categorical with missing filled as 'Unknown'.


<ipython-input-1-465ce396b91b>:76: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='raise')
<ipython-input-1-465ce396b91b>:76: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='raise')
<ipython-input-1-465ce396b91b>:76: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='raise')


Missing values after cleaning:
index                 0
Order ID              0
Date                  0
Status                0
Fulfilment            0
Sales Channel         0
ship-service-level    0
Style                 0
SKU                   0
Category              0
Size                  0
ASIN                  0
Courier Status        0
Qty                   0
currency              0
Amount                0
ship-city             0
ship-state            0
ship-postal-code      0
ship-country          0
promotion-ids         0
B2B                   0
fulfilled-by          0
Unnamed: 22           0
dtype: int64

--- Groq API Cleaning Instructions ---
Data Cleaning and Transformation Steps

Based on the provided data summary, the following steps are recommended to clean and transform the dataset for visualization in Power BI:

### Step 1: Handle Missing Values

*   Check for missing values in the dataset using the `isnull()` or `isna()` function in Python.
*   Decide on a strategy to h